In [1]:
import os
import pathlib
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import  Model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import *

In [2]:
def load_images_and_labels(image_paths):
    images = []
    labels = []

    for image_path in image_paths:
        image = load_img(image_path, target_size=(32,32),
                         color_mode='grayscale')
        image = img_to_array(image)

        label = image_path.split(os.path.sep)[-2]
        label = 'positive' in label
        label = float(label)

        images.append(image)
        labels.append(label)
    return np.array(images), np.array(labels)


In [3]:
def build_network():
    input_layer = Input(shape=(32,32,1))
    x = Conv2D(filters=20, kernel_size=(5,5), padding='same', strides=(1,1))(input_layer)
    x = ELU()(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
    x = Dropout(0.4)(x)

    x = Conv2D(filters=50, kernel_size=(5,5), padding='same', strides=(1,1))(x)
    x = ELU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2), strides=(2,2))(x)
    x = Dropout(0.4)(x)

    x = Flatten()(x)
    x = Dense(units=500)(x)
    x = ELU()(x)
    x = Dropout(0.4)(x)

    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_layer, outputs=output)
    return model


In [4]:
files_pattern = ('C:/Users/hp/Documents/DATA/SMILEsmileD-master/SMILEs/*/*/*.jpg')
files_pattern = str(files_pattern)
dataset_paths = [*glob.glob(files_pattern)]

In [5]:
X, y = load_images_and_labels(dataset_paths)

In [6]:
X /= 255.0
total = len(y)
total_positive = np.sum(y)
total_negative = total - total_positive

In [7]:
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.2, stratify=y, random_state=999)
(X_train, X_val, y_train,y_val) = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=999)

In [8]:
model = build_network()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
BATCH_SIZE = 32
EPOCHS = 20
model.fit(X_train, y_train, validation_data=(X_val, y_val),
          epochs = EPOCHS,
          batch_size=BATCH_SIZE,
          class_weight={1.0: total / total_positive,
                        0.0: total / total_negative})

Epoch 1/20
264/264 [==============================] - 17s 61ms/step - loss: 1.4306 - accuracy: 0.7624 - val_loss: 1.2630 - val_accuracy: 0.5653
Epoch 2/20
264/264 [==============================] - 18s 69ms/step - loss: 0.8321 - accuracy: 0.8273 - val_loss: 0.3863 - val_accuracy: 0.8424
Epoch 3/20
264/264 [==============================] - 16s 60ms/step - loss: 0.7166 - accuracy: 0.8545 - val_loss: 0.6096 - val_accuracy: 0.8624
Epoch 4/20
264/264 [==============================] - 17s 64ms/step - loss: 0.6606 - accuracy: 0.8611 - val_loss: 0.3749 - val_accuracy: 0.8609
Epoch 5/20
264/264 [==============================] - 18s 69ms/step - loss: 0.6264 - accuracy: 0.8764 - val_loss: 0.2534 - val_accuracy: 0.8946
Epoch 6/20
264/264 [==============================] - 18s 69ms/step - loss: 0.5638 - accuracy: 0.8856 - val_loss: 0.3478 - val_accuracy: 0.8628
Epoch 7/20
264/264 [==============================] - 18s 67ms/step - loss: 0.5325 - accuracy: 0.8919 - val_loss: 0.2980 - val_accuracy:

In [10]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

83/83 [==============================] - 1s 16ms/step - loss: 0.2463 - accuracy: 0.9096
